# Diffusion-Shock Inpainting in $\mathbb{R}^2$
Diffusion-shock inpainting (DS) is a technique to fill in missing structures in images, developed in ["Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://link.springer.com/chapter/10.1007/978-3-031-31975-4_45) and the follow-up paper ["Regularised Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://arxiv.org/abs/2309.08761). In this notebook, we will look at DS in the same setting as considered in these two papers, namely applied to images on $\mathbb{R}^2$. We will subsequently attempt to extend it to orientation scores in $SE(2)$ (maybe even $SO(3)$ eventually?).

We can describe DS in a PDE-based formulation as
$$
\partial_t u = g(\lvert \nabla (G_{\nu} * u) \rvert^2) \underbrace{\Delta u}_{\textrm{Diffusion}} - \left(1 - g(\lvert \nabla (G_{\nu} * u) \rvert^2)\right) \underbrace{\mathrm{sgn}(\partial_{\vec{w} \vec{w}} (G_{\sigma} * u)) \lvert \nabla u \rvert}_{\textrm{Shock}},
$$
in which $g: [0, \infty) \to (0, 1]$ is a decreasing function with $g(0) = 1$, $G_{\alpha}$ is a Gaussian with standard deviation $\alpha$, and $\vec{w}$ is the dominant eigenvector of the structure tensor. It is clear then that $g$ switches between applying diffusion and shock: if the gradient of the image is small, we mostly apply diffusion, but if the gradient is large, we mostly apply shock. This makes sense, since a large gradient implies that there is a feature there, which we would like to sharpen up. 

The signum in the shock term switches between erosion and dilation. If the second derivative with respect to the dominant eigenvector of the structure tensor is positive, then we perform erosion (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$); otherwise we perform dilation (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$). In regularised DS, the signum is replaced with a soft signum, so that the selection of erosion vs dilation is less sensitive to noise.

We will start by reproducing the algorithm described in ["Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://link.springer.com/chapter/10.1007/978-3-031-31975-4_45) on $\mathbb{R}^2$.

In [ ]:
import taichi as ti
ti.init(arch=ti.gpu, debug=False)
import numpy as np
# from PIL import Image
import matplotlib.pyplot as plt
# %matplotlib widget
import dsfilter

In [ ]:
test_case = 1

In [ ]:
match test_case:
    case 1: # Figure 2 (top) BIG
        dim_I, dim_J = 256, 256
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        u_ground_truth[:25, :] = 0.
        u_ground_truth[100:160, :] = 0.
        u_ground_truth[-25:, :] = 0.

        mask = np.ones((dim_I, dim_J))
        mask[:, 100:160] = 0.
        u = u_ground_truth.copy()
        u[:, 100:160] = 0.5 * 255.

        T_short = 1/3
        T_medium = 15
        T_long = 100
        T_mega_long = 1000

        σ = 3 # Internal regularisation of structure tensor for switching between dilation and erosion.
        ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
        ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
        λ = 3 # Contrast parameter for switching between diffusion and shock.
        ε = 0.15 * λ # Regularisation parameter for signum.
    case 2: # Figure 2 (top) small
        dim_I, dim_J = 128, 128
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        u_ground_truth[:12, :] = 0.
        u_ground_truth[50:80, :] = 0.
        u_ground_truth[-12:, :] = 0.

        mask = np.ones((dim_I, dim_J))
        mask[:, 50:80] = 0.
        u = u_ground_truth.copy()
        u[:, 50:80] = 0.5 * 255.

        T_short = 1/3
        T_medium = 15
        T_long = 100
        T_mega_long = 1000

        σ = 2 # Internal regularisation of structure tensor for switching between dilation and erosion.
        ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
        ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
        λ = 3 # Contrast parameter for switching between diffusion and shock.
        ε = 0.15 * λ # Regularisation parameter for signum.
    case 3: # Figure 3 (top)
        dim_I, dim_J = 128, 128
        u_ground_truth = np.ones((dim_I, dim_J)) * 0.5 * 255.
        u_ground_truth[:65, :] = 0.
        u_ground_truth[65:, :] = 1. * 255.

        u = np.ones((dim_I, dim_J)) * 0.5 * 255.
        u[64, 65] = 0.
        u[65, 65] = 1. * 255.
        mask = np.zeros((dim_I, dim_J))
        mask[64:66, 65] = 1.

        T_short = 1/3
        T_medium = 100
        T_long = 1000
        T_mega_long = 10000

        σ = 2 # Internal regularisation of structure tensor for switching between dilation and erosion.
        ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
        ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
        λ = 1 # Contrast parameter for switching between diffusion and shock.
        ε = 0.15 * λ # Regularisation parameter for signum.
    case 4: # Figure 2 (bottom)
        dim_I, dim_J = 256, 256
        u_ground_truth = np.zeros((dim_I, dim_J))
        u_ground_truth[:, 100:160] = 1. * 255.
        u_ground_truth[100:160, :] = 1. * 255.

        mask = np.ones((dim_I, dim_J))
        mask[80:180, 80:180] = 0.
        u = u_ground_truth.copy()
        u[80:180, 80:180] = 0.5 * 255

        T_short = 1/3
        T_medium = 50
        T_long = 250
        T_mega_long = 2000

        σ = 2 # Internal regularisation of structure tensor for switching between dilation and erosion.
        ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
        ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
        λ = 3 # Contrast parameter for switching between diffusion and shock.
        ε = 0.15 * λ # Regularisation parameter for signum.
    case 5: # Figure 8 (top) of RDS
        dim_I, dim_J = 256, 256
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        u_ground_truth = ((xs**2 + ys**2) < 0.8) * 255.

        mask = np.ones((dim_I, dim_J))
        mask[128:250, 5:128] = 0.
        u = u_ground_truth.copy()
        u[128:250, 5:128] = 0.5 * 255

        T_short = 1/3
        T_medium = 100
        T_long = 1000
        T_mega_long = 10000

        σ = 2.5 # Internal regularisation of structure tensor for switching between dilation and erosion.
        ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
        ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
        λ = 2 # Contrast parameter for switching between diffusion and shock.
        ε = 0.15 * λ # Regularisation parameter for signum.
    case 6: # Figure 4
        dim_I, dim_J = 256, 256
        xs, ys = np.meshgrid(np.linspace(-1., 1., dim_I), np.linspace(-1., 1., dim_J), indexing="ij")
        u_ground_truth = (ys > 0.700 * xs - 0.487) * (ys > -3.53 * xs - 1.77) * (ys < -0.502 * xs + 0.349) * 255.

        radius = np.sqrt(0.0426)
        mask = (((xs + 0.303)**2 + (ys + 0.699)**2 < radius**2) +
                ((xs - 0.695)**2 + ys**2 < radius**2) +
                ((xs + 0.699)**2 + (ys - 0.701)**2 < radius**2)) * 1.
        u = np.random.random((dim_I, dim_J)) * 255. * (1. - mask) + u_ground_truth * mask

        T_short = 1/3
        T_medium = 100
        T_long = 1000
        T_mega_long = 10000

        σ = 3.5 # Internal regularisation of structure tensor for switching between dilation and erosion.
        ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
        ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
        λ = 3. # Contrast parameter for switching between diffusion and shock.
        ε = 0.15 * λ # Regularisation parameter for signum.

dxy = 1.
x_min, x_max = 0, u.shape[0] - 1
y_min, y_max = 0, u.shape[1] - 1

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("$u$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("Mask")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
u_filtered_short, switch_DS_short, switch_morph_short = dsfilter.DS_filter_R2(u, mask, T_short, σ, ρ, ν, λ, ε=ε, dxy=dxy)
u_filtered_medium, switch_DS_medium, switch_morph_medium = dsfilter.DS_filter_R2(u, mask, T_medium, σ, ρ, ν, λ, ε=ε, dxy=dxy)
u_filtered_long, switch_DS_long, switch_morph_long = dsfilter.DS_filter_R2(u, mask, T_long, σ, ρ, ν, λ, ε=ε, dxy=dxy)
u_filtered_mega_long, switch_DS_mega_long, switch_morph_mega_long = dsfilter.DS_filter_R2(u, mask, T_mega_long, σ, ρ, ν, λ, ε=ε, dxy=dxy)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_short, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_medium, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_mega_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_medium, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_mega_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax);